# Train U-Net Segmentation
This can be run in google Colab to demo training. Adapted from DL4MIA 2022 Unet.
https://github.com/dl4mia.

**HOWEVER I HAVE NOT TESTED THIS FOR OCT SEGMENTATION IN COLAB**

## Initialize Colab environment
Clone the repo, install the packages, download the data, and ensure torch available.

In [ ]:
# clone my notebook
!git clone https://github.com/obria006/unet_segmentation.git

In [ ]:
# change to the cloned unet repo
%cd unet_segmentation
!pip install torch torchvision
!pip install -e.

In [ ]:
# download data
import os
import urllib.request
import zipfile
import shutil
def extract_data(zip_src, dst):
    """
        Extracts data in zip file at 'zip_src' to the 'dst' directory.
        ----------
        zip_src: string
            path to zip file
        dst: string
            Indicates the path to the directory where the data should be saved.

    """
    zip_name = os.path.basename(zip_src)
    zip_name_no_ext = zip_name.strip(".zip")
    zip_path = os.path.join(dst, zip_name)

    if (os.path.exists(os.path.join(dst, zip_name_no_ext))):
        print(f"Destination directory already exists at: {os.path.abspath(os.path.join(dst, zip_name_no_ext))}")
    else:
        os.makedirs(os.path.join(dst, zip_name_no_ext))
        with zipfile.ZipFile(zip_src, 'r') as zip_ref:
            zip_ref.extractall(dst)

extract_data(
    zip_src = f"{os.getcwd()}/data/export/OCT_scans_128x128.zip",
    dst = f"{os.getcwd()}/data/processed"
)

## Display network training hardware
This will display whether the training device is GPU (cuda) or CPU. GPU will train the network substantially faster than CPU.

In [ ]:
# setting device on GPU if available, else CPU
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
print()

#Additional Info when using cuda
if device.type == 'cuda':
    print(torch.cuda.get_device_name(0))
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_reserved(0)/1024**3,1), 'GB')

## Start training

At a minimum, you must specify the:
1. Data directory (`data_dir`) which is the directory where the "train", "test", and "val" folders are located that contain processed images/masks.
2. Output/results directory (`output_dir`) which is the directory where the training results will be saved. Within this directory, folders for each training session will be created.

Addtionally, you can view the `main()` function parameters in the "train.py" file to see other keyword arguments that can be specified. For instance, you can change the number of training epochs, number of layers in the U-Net, and the size to which input images will be resized prior to training.

In [ ]:
from src.models.dl4mia_tissue_unet import train
data_dir = "../data/processed/OCT_scans_128x128"
output_dir = "../src/models/dl4mia_tissue_unet/results"
n_epochs = 30
train.main(data_dir=data_dir, output_dir = output_dir, n_epochs=n_epochs)

## Predictions

In [ ]:
%load_ext autoreload
%autoreload 2
from src.models.dl4mia_tissue_unet import predict
import glob
res_dirs = glob.glob(f"{output_dir}/*")
if res_dirs != []:
  result_dir = res_dirs[-1]
  ckpt_name = "best.pth"
  deploy_dir = "../data/OCT_scans/raw/images"
  predict.main(src_dir=result_dir, ckpt_name=ckpt_name, deploy_dir=None)

## Download results


In [ ]:
!zip -r /content/results.zip /content/unet_segmentation/results
from google.colab import files
files.download("/content/results.zip")